In [1]:
# Author  : Karthikeyan Rasipalayam Durairaj 
# Dataset : HTTP requests to the NASA Kennedy Space Center—is freely available 
#https://www.kaggle.com/gagandeep16/car-sales

In [2]:
# import packages
from pyspark.sql import SparkSession, HiveContext, SQLContext
from pyspark.sql.functions import when, desc
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import to_date, trunc, unix_timestamp, date_format
from pyspark.sql.window import Window
from datetime import datetime
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import regexp_extract
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.functions import count, col 
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
 


In [3]:
base_df = spark.read.csv("dbfs:/FileStore/tables/Car_sales.csv",header = True)


In [4]:
base_df.printSchema()

root
-- Manufacturer: string (nullable = true)
-- Model: string (nullable = true)
-- Sales_in_thousands: string (nullable = true)
-- __year_resale_value: string (nullable = true)
-- Vehicle_type: string (nullable = true)
-- Price_in_thousands: string (nullable = true)
-- Engine_size: string (nullable = true)
-- Horsepower: string (nullable = true)
-- Wheelbase: string (nullable = true)
-- Width: string (nullable = true)
-- Length: string (nullable = true)
-- Curb_weight: string (nullable = true)
-- Fuel_capacity: string (nullable = true)
-- Fuel_efficiency: string (nullable = true)
-- Latest_Launch: string (nullable = true)
-- Power_perf_factor: string (nullable = true)

In [5]:
base_df.show()

+------------+-----------+------------------+-------------------+------------+------------------+-----------+----------+---------+-----+------+-----------+-------------+---------------+-------------+-----------------+
Manufacturer| Model|Sales_in_thousands|__year_resale_value|Vehicle_type|Price_in_thousands|Engine_size|Horsepower|Wheelbase|Width|Length|Curb_weight|Fuel_capacity|Fuel_efficiency|Latest_Launch|Power_perf_factor|
+------------+-----------+------------------+-------------------+------------+------------------+-----------+----------+---------+-----+------+-----------+-------------+---------------+-------------+-----------------+
 Acura| Integra| 16.919| 16.36| Passenger| 21.5| 1.8| 140| 101.2| 67.3| 172.4| 2.639| 13.2| 28| 2/2/2012| 58.28014952|
 Acura| TL| 39.384| 19.875| Passenger| 28.4| 3.2| 225| 108.1| 70.3| 192.9| 3.517| 17.2| 25| 6/3/2011| 91.37077766|
 Acura| CL| 14.114| 18.225| Passenger| null| 3.2| 225| 106.9| 70.6| 192| 3.47| 17.2| 26| 1/4/2012| null|
 Acura| RL| 8.588| 29.725| Passenger| 42| 3.5| 210| 114.6| 71.4| 196.6| 3.85| 18| 22| 3/10/2011| 91.38977933|
 Audi| A4| 20.397| 22.255| Passenger| 23.99| 1.8| 150| 102.6| 68.2| 178| 2.998| 16.4| 27| 10/8/2011| 62.7776392|
 Audi| A6| 18.78| 23.555| Passenger| 33.95| 2.8| 200| 108.7| 76.1| 192| 3.561| 18.5| 22| 8/9/2011| 84.56510502|
 Audi| A8| 1.38| 39| Passenger| 62| 4.2| 310| 113| 74| 198.2| 3.902| 23.7| 21| 2/27/2012| 134.6568582|
 BMW| 323i| 19.747| null| Passenger| 26.99| 2.5| 170| 107.3| 68.4| 176| 3.179| 16.6| 26| 6/28/2011| 71.19120671|
 BMW| 328i| 9.231| 28.675| Passenger| 33.4| 2.8| 193| 107.3| 68.5| 176| 3.197| 16.6| 24| 1/29/2012| 81.87706856|
 BMW| 528i| 17.527| 36.125| Passenger| 38.9| 2.8| 193| 111.4| 70.9| 188| 3.472| 18.5| 25| 4/4/2011| 83.9987238|
 Buick| Century| 91.561| 12.475| Passenger| 21.975| 3.1| 175| 109| 72.7| 194.6| 3.368| 17.5| 25| 11/2/2011| 71.18145132|
 Buick| Regal| 39.35| 13.74| Passenger| 25.3| 3.8| 240| 109| 72.7| 196.2| 3.543| 17.5| 23| 9/3/2011| 95.63670253|
 Buick|Park Avenue| 27.851| 20.19| Passenger| 31.965| 3.8| 205| 113.8| 74.7| 206.8| 3.778| 18.5| 24| 3/23/2012| 85.82840825|
 Buick| LeSabre| 83.257| 13.36| Passenger| 27.885| 3.8| 205| 112.2| 73.5| 200| 3.591| 17.5| 25| 7/23/2011| 84.25452581|
 Cadillac| DeVille| 63.729| 22.525| Passenger| 39.895| 4.6| 275| 115.3| 74.5| 207.2| 3.978| 18.5| 22| 2/23/2012| 113.8545976|
 Cadillac| Seville| 15.943| 27.1| Passenger| 44.475| 4.6| 275| 112.2| 75| 201| null| 18.5| 22| 4/29/2011| 115.6213578|
 Cadillac| Eldorado| 6.536| 25.725| Passenger| 39.665| 4.6| 275| 108| 75.5| 200.6| 3.843| 19| 22| 11/27/2011| 113.7658739|
 Cadillac| Catera| 11.185| 18.225| Passenger| 31.01| 3| 200| 107.4| 70.3| 194.8| 3.77| 18| 22| 9/28/2011| 83.48309358|
 Cadillac| Escalade| 14.785| null| Car| 46.225| 5.7| 255| 117.5| 77| 201.2| 5.572| 30| 15| 4/17/2012| 109.5091165|
 Chevrolet| Cavalier| 145.519| 9.25| Passenger| 13.26| 2.2| 115| 104.1| 67.9| 180.9| 2.676| 14.3| 27| 8/17/2011| 46.36334747|
+------------+-----------+------------------+-------------------+------------+------------------+-----------+----------+---------+-----+------+-----------+-------------+---------------+-------------+-----------------+
only showing top 20 rows

In [6]:
w_all = Window().partitionBy().orderBy(col("Sales_in_thousands").desc())

(base_df
  .withColumn("rn", row_number().over(w_all))
  .where(col("rn") == 1)
  .select("Manufacturer", "Model","Sales_in_thousands")).show()

+------------+----------+------------------+
Manufacturer| Model|Sales_in_thousands|
+------------+----------+------------------+
 Pontiac|Grand Prix| 92.364|
+------------+----------+------------------+

In [7]:


w = Window().partitionBy("Manufacturer").orderBy(col("Sales_in_thousands").desc())

(base_df
  .withColumn("rn", row_number().over(w))
  .where(col("rn") == 1)
  .select("Manufacturer", "Model","Sales_in_thousands")).show()


                                                

+------------+-------------+------------------+
Manufacturer| Model|Sales_in_thousands|
+------------+-------------+------------------+
 Volkswagen| Golf| 9.761|
 Oldsmobile| Alero| 80.255|
 Infiniti| I30| 23.713|
 Lexus| LX470| 9.126|
 Jaguar| S-Type| 15.467|
 Saturn| SL| 80.62|
 Jeep| Cherokee| 80.556|
 Mitsubishi| Galant| 55.616|
 Chevrolet| Monte Carlo| 42.593|
 Volvo| V40| 3.545|
 Hyundai| Elantra| 66.692|
 Saab| 5-Sep| 9.191|
 Honda| Odyssey| 76.029|
 Audi| A4| 20.397|
 Cadillac| DeVille| 63.729|
 Plymouth| Breeze| 5.24|
 Pontiac| Grand Prix| 92.364|
 Mercury|Grand Marquis| 81.174|
 Chrysler|Sebring Coupe| 7.854|
 Lincoln| Town car| 48.911|
+------------+-------------+------------------+
only showing top 20 rows